In [1]:
!python --version

Python 3.6.5 :: Anaconda, Inc.


In [2]:
#!pip install python-decouple

In [3]:
#!pip install nest_asyncio

In [11]:
#!pip install aiohttp

In [45]:
#!pip install blackcellmagic

In [46]:
%load_ext blackcellmagic

In [2]:
#!pip install python-decouple

In [2]:
from decouple import config
import json
import requests
import sys
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
from itertools import zip_longest
import aiohttp
from aiohttp import ClientSession
import pandas as pd
from random import randint
nest_asyncio.apply()
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import re

ModuleNotFoundError: No module named 'decouple'

In [5]:
CLIENT_SECRET= config("CLIENT_SECRET")
CLIENT_ID = config("CLIENT_ID")
CLIENT_ACCESS_TOKEN = config("CLIENT_ACCESS_TOKEN")

In [15]:
#max_song = 2110000
test_song =100

## Step One: Get all the URL's for each song

In [20]:
import requests


def request_song_info(session, song_num, song_urls):
    base_url = "https://api.genius.com/songs/" + str(song_num)
    headers = {"Authorization": "Bearer " + CLIENT_ACCESS_TOKEN}
    response = requests.get(base_url, headers=headers)
    try:
        if response.json()["meta"]["status"] == 200:
            song_urls.append(
                (
                    song_num,
                    response.json()["response"]["song"]["title"],
                    response.json()["response"]["song"]["url"],
                )
            )
        else:
            pass
    except:
        pass

In [29]:
async def get_index_data_asynchronous(min_song, max_song, song_urls):
    """
    1. Establish an executor and number of workers
    2. Establish the session
    3. Establish the event loop
    4. Create the task by list comprenhensions
    5. Gather tasks.
    """
    with ThreadPoolExecutor(max_workers=200) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, request_song_info, *(session, song_num, song_urls)
                )
                for song_num in range(min_song, max_song)
            ]
            for response in await asyncio.gather(*tasks):
                pass


def execute_async_index_event_loop(min_song, max_song, song_urls):
    """
    This function does something analogous to compiling the get_data_asynchronously function,
    Then it executes loop.
    1. Call the get_data_function
    2. Get the event_loop
    3. Run the tasks (Much easier to understand in python 3.7, "ensure_future" was changed to "create_task")
    4. Edge_list and top_interactions will be passed to the next functions
    """
    future = asyncio.ensure_future(
        get_index_data_asynchronous(min_song, max_song, song_urls)
    )
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    return song_urls

## tests

In [10]:
%%time
song_urls = []
min_song=0
test_song_urls = execute_async_index_event_loop(min_song,test_song,song_urls)

Wall time: 955 ms


In [11]:
len(test_song_urls)

91

## Running this iteratively
It takes about ~15 minutes per 100,000 queries. This algorithm does have poor error handling(it just skips some errors, but some raise exception and causes it to break), so the total ends up less than is expected. However, from working on this project a lot I know that many entries have been deleted and should explain the missing values.

In [24]:
song_urls = []

In [104]:
%%time
min_song = max([s[0] for s in song_urls])+1
max_song = min_song +200000
song_urls = execute_async_index_event_loop(min_song, max_song,song_urls)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [109]:
len(song_urls)

1917750

## Pickling. 
Pickling the data after each loop.

In [110]:
import pickle
filename = "song_list_tuple"
outfile = open(filename,'wb')
pickle.dump(song_urls, outfile)
outfile.close()

In [6]:
import pickle
filename = "song_list_tuple"
infile = open(filename, "rb")
song_urls = pickle.load(infile)
infile.close()

In [7]:
song_urls

[(52, 'Niggaz', 'https://genius.com/Fabolous-niggaz-lyrics'),
 (39, 'Fireman', 'https://genius.com/Lil-wayne-fireman-lyrics'),
 (13,
  'Think Ya’ll Know',
  'https://genius.com/Fabolous-think-yall-know-lyrics'),
 (57,
  'Back That Azz Up',
  'https://genius.com/Juvenile-back-that-azz-up-lyrics'),
 (43, 'Juicy', 'https://genius.com/The-notorious-big-juicy-lyrics'),
 (23, 'Fly Out', 'https://genius.com/Lil-wayne-fly-out-lyrics'),
 (16, '’98 Freestyle', 'https://genius.com/Big-l-98-freestyle-lyrics'),
 (22, 'Broads', 'https://genius.com/Big-tymers-broads-lyrics'),
 (41,
  'Whatever I Wan’ Do',
  'https://genius.com/Fabolous-whatever-i-wan-do-lyrics'),
 (21, 'Colors', 'https://genius.com/Gucci-mane-colors-lyrics'),
 (11, 'Lord You Know', 'https://genius.com/Camron-lord-you-know-lyrics'),
 (35, 'I’m a Hustla', 'https://genius.com/Re-up-gang-im-a-hustla-lyrics'),
 (49,
  'I’m Gettin’ Money',
  'https://genius.com/Fabolous-im-gettin-money-lyrics'),
 (83, 'Mrs. Officer', 'https://genius.com/Li

In [22]:
len(song_urls)

1917750

### Putting it all together

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import re


def get_page(session,song_tuple,songs_list):
    url = song_tuple[2]
    try:
        page = requests.get(url)
        t = get_text(page)
        lyrics = get_lyrics(t)
        table = get_side_table_rows(t)
        header = get_header_links(t)
        songs_list.append((song_tuple[0],song_tuple[1],song_tuple[2],lyrics, table, header))
    except:
        pass
    

In [9]:
def get_text(response):
    """Get text from a requests response object"""
    r = response.text
    r = BeautifulSoup(r, "html.parser")
    return r


def get_lyrics(soup_text):
    """extract lyrics from the beautiful soup text"""
    try:
        lyrics = soup_text.find_all("p")[0].text
        lyrics = re.sub(r"\[.*?\']", "", lyrics)
        lyrics = re.sub(r"--", "", lyrics)
        lyrics = lyrics.split("\n")
        lyrics = [x for x in lyrics if x != ""]
        return lyrics
    except:
        np.nan


def get_side_table_rows(page):
    """extract the side tables from the beatiful soup text"""
    side_table = page.find_all(
        "div", {"class": "metadata_unit metadata_unit--table_row"}
    )
    side_table = [get_side_table_row(i) for i in side_table]
    return side_table


def get_side_table_row(side_list_table_row):
    a = side_list_table_row.find_all("span", {"class":"metadata_unit-label"})
    label = a[0].text
    links_list = side_list_table_row.find_all("a")
    links = [get_links(links_list,i) for i in range(len(links_list))]
    return label,links

def get_links(x, i):
    return x[i]["href"]

def get_header_links(page):
    """
    find unit containing the header info
    use 
    """
    header = page.findAll("div", {"class":"header_with_cover_art-primary_info" })
    header_links = [i.a["href"] for i in header ]
    return header_links

In [15]:
async def get_index_data_asynchronous(songs_list, song_urls):
    """
    1. Establish an executor and number of workers
    2. Establish the session
    3. Establish the event loop
    4. Create the task by list comprenhensions
    5. Gather tasks.
    """
    with ThreadPoolExecutor(max_workers=40) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, get_page, *(session, song_tuple,songs_list)
                )
                for song_tuple in song_urls
            ]
            for response in await asyncio.gather(*tasks):
                pass


def execute_async_index_event_loop(songs_list, song_urls,i):
    """
    This function does something analogous to compiling the get_data_asynchronously function,
    Then it executes loop.
    1. Call the get_data_function
    2. Get the event_loop
    3. Run the tasks (Much easier to understand in python 3.7, "ensure_future" was changed to "create_task")
    4. Edge_list and top_interactions will be passed to the next functions
    """
    future = asyncio.ensure_future(
        get_index_data_asynchronous(songs_list, song_urls)
    )
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    filename = "song_lists/songs_list"+str(i)
    outfile = open(filename,'wb')
    pickle.dump(songs_list1, outfile)
    outfile.close()

In [31]:
for i in range(len(os.listdir("song_lists/")),200,1):
    songs_list1=[]
    song_urls1 =song_urls[i*10000:i*10000+10000]
    execute_async_index_event_loop(songs_list1, song_urls1,i)

# extracting info from pages

In [35]:
import pickle
import os
pickle_list = os.listdir("song_lists/")

In [36]:
len(pickle_list)

199

In [37]:
pickle_list

['songs_list27',
 'songs_list130',
 'songs_list134',
 'songs_list179',
 'songs_list196',
 'songs_list86',
 'songs_list42',
 'songs_list115',
 'songs_list3720012',
 'songs_list23',
 'songs_list81',
 'songs_list29',
 'songs_list53',
 'songs_list162',
 'songs_list183',
 'songs_list68',
 'songs_list150',
 'songs_list26',
 'songs_list77',
 'songs_list43',
 'songs_list21',
 'songs_list92854519',
 'songs_list75',
 'songs_list197',
 'songs_list57',
 'songs_list89',
 'songs_list108',
 'songs_list69',
 'songs_list117',
 'songs_list163',
 'songs_list141',
 'songs_list188',
 'songs_list107',
 'songs_list60',
 'songs_list177',
 'songs_list119',
 'songs_list148',
 'songs_list138',
 'songs_list192',
 'songs_list139',
 'songs_list49',
 'songs_list97',
 'songs_list174',
 'songs_list82',
 'songs_list61',
 'songs_list19',
 'songs_list16',
 'songs_list155',
 'songs_list193',
 'songs_list126',
 'songs_list175',
 'songs_list24',
 'songs_list92',
 'songs_list111',
 'songs_list48',
 'songs_list91',
 'songs_li

In [38]:
filename = "song_lists/"+pickle_list[0]
outfile = open(filename,'rb')
z = pickle.load(outfile)
outfile.close()

In [50]:
z[1]

(358490,
 'Fuck Em All',
 'https://genius.com/Caution-fuck-em-all-lyrics',
 ['[Hook]',
  'You can catch me rollin wit my goonies we be wildin out forreal',
  'Yall be wildin out for pills we out here grindin for a deal',
  'Same niggas hatin on me same niggas showin love',
  "Ain't nobody fuckin wit me on everything that I love",
  'They ask me how I feel fuck em all',
  'How you feel ? fuck em all',
  "That's how you feel ? Fuck Em All",
  "That's how I feel fuck em all",
  'Same niggas hatin same niggas tryna be me',
  'Same niggas with they handout everytime them niggas see me fuck em',
  "I don't trust em why would I ever do that ?",
  "Fuck wit ya'll niggas fore I ever get my crew back",
  'Im talkin bout that mafia body ya keep your head stash',
  'Get the work bag it then move it and get the bread last',
  'Nigga now what you know bout that ?',
  "If you don't bout this,you don't know bout that",
  "If you don't know hot to flip, then you don't know how to stack",
  "Cuz once I 

In [56]:
df.append(z)

,song_num,title,url,lyrics,side_table,header_links
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[I've got nowhere left to hide, No time left f...","[(Release Date, [])]",[https://genius.com/artists/Dangerkids]
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"[[Hook], You can catch me rollin wit my goonie...","[(Release Date, [])]",[https://genius.com/artists/Caution]
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,[There is in Ghana story-feeling art called An...,"[(Release Date, [])]",[https://genius.com/artists/Freeman-dwamena]
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"[[Verse 1], You don't see me, Now I am far awa...","[(Release Date, [])]",[https://genius.com/artists/Kazaky]
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"[[Verse 1 - Jimmy The Rapper], Came came, saw ...","[(Release Date, [])]",[https://genius.com/artists/Jimmy-the-rapper-a...
5,358482,Coal Town Road,https://genius.com/Ryans-fancy-coal-town-road-...,"[[Chorus], We get up in the black down the coa...","[(Release Date, [])]",[https://genius.com/artists/Ryans-fancy]
6,358477,Tworzymy Historię,https://genius.com/Jwp-bc-tworzymy-historie-ly...,"[[Verse 1: Ero/Kosi/Łysol], Światła, kamera, a...","[(Written By, [https://genius.com/artists/Ajzo...",[https://genius.com/artists/Jwp-bc]
7,358503,Paradiso: Canto 29,https://genius.com/Dante-alighieri-paradiso-ca...,"[Paradiso: Canto XXIX, At what time both the c...",[],[https://genius.com/artists/Dante-alighieri]
8,358506,Filly Dreams,https://genius.com/Jeff-burgess-and-the-bad-ma...,"[Are you scared of the way things change, When...","[(Release Date, [])]",[https://genius.com/artists/Jeff-burgess-and-t...
9,358485,Never Forget (5 Years Later),https://genius.com/Lozo-never-forget-5-years-l...,[We're constantly emboldened to neVer forgeT 3...,[],[https://genius.com/artists/Lozo]


In [39]:
df = pd.DataFrame(columns=["song_num","title","url","lyrics","side_table","header_links"])
for i in pickle_list:
    #load the pickled tuple and convert to df
    filename = "song_lists/"+i
    outfile = open(filename,'rb')
    z = pickle.load(outfile)
    z = pd.DataFrame(z, columns =["song_num","title","url","lyrics","side_table","header_links"])
    df = df.append(z)
    outfile.close()

In [40]:
df.shape

(1907750, 6)

In [41]:
df.to_csv("million.9_song.csv", index=False)

## Creating the model

In [17]:
import pandas as pd
from ast import literal_eval

In [18]:
df= pd.read_csv("million.9_song.csv")

AttributeError: module 'pandas' has no attribute 'read_csv'

In [4]:
df.shape

(1844757, 12)

In [6]:
df.dropna(inplace=True)

In [7]:
df.drop_duplicates(subset=["url"],inplace=True)

In [8]:
df.isna().sum()

song_num        0
title           0
url             0
lyrics          0
side_table      0
header_links    0
dtype: int64

In [9]:
df.shape

(1906918, 6)

In [10]:
from collections import Counter
Counter(df["url"]).most_common(50)

[('https://genius.com/Dangerkids-hostage-lyrics', 1),
 ('https://genius.com/Caution-fuck-em-all-lyrics', 1),
 ('https://genius.com/Freeman-dwamena-ananseem-stories-annotated', 1),
 ('https://genius.com/Kazaky-the-sun-lyrics', 1),
 ('https://genius.com/Jimmy-the-rapper-and-jay-bel-amused-lyrics', 1),
 ('https://genius.com/Ryans-fancy-coal-town-road-lyrics', 1),
 ('https://genius.com/Jwp-bc-tworzymy-historie-lyrics', 1),
 ('https://genius.com/Dante-alighieri-paradiso-canto-29-annotated', 1),
 ('https://genius.com/Jeff-burgess-and-the-bad-mares-filly-dreams-lyrics', 1),
 ('https://genius.com/Lozo-never-forget-5-years-later-annotated', 1),
 ('https://genius.com/Dante-alighieri-inferno-canto-5-mandelbaum-annotated',
  1),
 ('https://genius.com/Ro-james-indiana-jones-lyrics', 1),
 ('https://genius.com/Jeff-burgess-and-the-bad-mares-i-cant-live-like-this-anymore-lyrics',
  1),
 ('https://genius.com/Jeff-burgess-and-the-bad-mares-vinyls-dream-lyrics', 1),
 ('https://genius.com/William-shakespe

In [49]:
df.to_csv("million.9_song.csv", index=False)

In [50]:
df.head()

,song_num,title,url,lyrics,side_table,header_links
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids']
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution']
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena']
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky']
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...


In [12]:
type(df["side_table"][0])

str

In [11]:
def list_check(i):
    if type(i) == list:
        return i

def extract_links(list_of_tuples):
    """
    1. List of tuples was saved as a string, turn it back into a list
    2. Turn Tuples into list. Output is a list of strings & lists
    3. filter only the lists, drop the identifiers such as "Written by". Output is a list of lists. 
    4. Turn the list of lists into one list
        4a. Remove stuff from the url stuff from the song url to get an identifier.
    """
    a = literal_eval(list_of_tuples)
    a = list(filter(list_check, [i[1] for i in a]))
    a = [
        j.replace("https://genius.com/artists/", "")
        .replace("https://genius.com/", "")
        .replace("artists/", "")
        .replace("-lyrics", "")
        .lower()
        for i in a
        for j in i
    ]
    return a
def get_header_links(item):
    """
    1. List was saved as a string. turn it back into a list
    2. 
    """
    item = literal_eval(item)
    if (type(item) == list) &( len(item)>0):
        item = item[0].replace("https://genius.com/artists/", "")
        item = item.lower()
        return [item]

In [12]:
df["meta_data_1"]=df.side_table.apply(extract_links)

In [13]:
df.head()

,song_num,title,url,lyrics,side_table,header_links,meta_data_1
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids'],[]
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution'],[]
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena'],[]
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky'],[]
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...,[]


In [14]:
df["fixed_header_links"] =  df.header_links.apply(get_header_links)

In [15]:
def return_longest(a,b):
    if (type(a)==list) & (type(b)==list):
        if len(a)>len(b):
            return a
        else:
            return b
    elif type(b)!=list:
        return a

In [16]:
import numpy as np

In [17]:
df["meta_data_2"] = np.array([return_longest(a,b) for a,b in zip(df["meta_data_1"],df["fixed_header_links"])])

In [18]:
df["song_identifier"] = (
    df.url.str.replace("https://genius.com/", "")
    .str.replace("-lyrics", "")
    .str.replace("https://genius.com/artists/", "")
    .str.lower()
)

In [19]:
df.head()

,song_num,title,url,lyrics,side_table,header_links,meta_data_1,fixed_header_links,meta_data_2,song_identifier
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids'],[],[dangerkids],[dangerkids],dangerkids-hostage
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution'],[],[caution],[caution],caution-fuck-em-all
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena'],[],[freeman-dwamena],[freeman-dwamena],freeman-dwamena-ananseem-stories-annotated
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky'],[],[kazaky],[kazaky],kazaky-the-sun
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...,[],[jimmy-the-rapper-and-jay-bel],[jimmy-the-rapper-and-jay-bel],jimmy-the-rapper-and-jay-bel-amused


## Lyric embeddings

In [20]:
import re


def clean_lyrics(lyrics):
    """
    1. turn the pickled list into into an actual list
    2. rejoin the list into one string
    3. regex and other string operations to clean the lyrics
    """
    lyrics = literal_eval(lyrics)
    l = " ".join(lyrics)
    l = re.sub(r'\[(.+?)\]|"', " ", l)
    return (
        l.replace("'", "")
        .replace(r", ", " ")
        .replace("(", "")
        .replace(")", "")
        .replace("?", "")
        .replace(":" "", "")
        .strip()
        .lower()
    )

In [21]:
df["clean_lyrics"] = df.lyrics.apply(clean_lyrics)

In [22]:
df.head()

,song_num,title,url,lyrics,side_table,header_links,meta_data_1,fixed_header_links,meta_data_2,song_identifier,clean_lyrics
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids'],[],[dangerkids],[dangerkids],dangerkids-hostage,ive got nowhere left to hide no time left for ...
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution'],[],[caution],[caution],caution-fuck-em-all,you can catch me rollin wit my goonies we be w...
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena'],[],[freeman-dwamena],[freeman-dwamena],freeman-dwamena-ananseem-stories-annotated,there is in ghana story-feeling art called ana...
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky'],[],[kazaky],[kazaky],kazaky-the-sun,you dont see me now i am far away you dont fee...
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...,[],[jimmy-the-rapper-and-jay-bel],[jimmy-the-rapper-and-jay-bel],jimmy-the-rapper-and-jay-bel-amused,came came saw saw conquered your body.. frame ...


In [6]:
!pip install gensim

    100% |████████████████████████████████| 24.2MB 2.0MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 32.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built smart-open
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
df["lyric_list"] =df["clean_lyrics"].str.split(" ")

In [25]:
df.head()

,song_num,title,url,lyrics,side_table,header_links,meta_data_1,fixed_header_links,meta_data_2,song_identifier,clean_lyrics,lyric_list
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids'],[],[dangerkids],[dangerkids],dangerkids-hostage,ive got nowhere left to hide no time left for ...,"[ive, got, nowhere, left, to, hide, no, time, ..."
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution'],[],[caution],[caution],caution-fuck-em-all,you can catch me rollin wit my goonies we be w...,"[you, can, catch, me, rollin, wit, my, goonies..."
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena'],[],[freeman-dwamena],[freeman-dwamena],freeman-dwamena-ananseem-stories-annotated,there is in ghana story-feeling art called ana...,"[there, is, in, ghana, story-feeling, art, cal..."
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky'],[],[kazaky],[kazaky],kazaky-the-sun,you dont see me now i am far away you dont fee...,"[you, dont, see, me, now, i, am, far, away, yo..."
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...,[],[jimmy-the-rapper-and-jay-bel],[jimmy-the-rapper-and-jay-bel],jimmy-the-rapper-and-jay-bel-amused,came came saw saw conquered your body.. frame ...,"[came, came, saw, saw, conquered, your, body....."


In [26]:
df = df[df["clean_lyrics"].str.len()>50]

In [27]:
df.reset_index()

,index,song_num,title,url,lyrics,side_table,header_links,meta_data_1,fixed_header_links,meta_data_2,song_identifier,clean_lyrics,lyric_list
0,0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids'],[],[dangerkids],[dangerkids],dangerkids-hostage,ive got nowhere left to hide no time left for ...,"[ive, got, nowhere, left, to, hide, no, time, ..."
1,1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution'],[],[caution],[caution],caution-fuck-em-all,you can catch me rollin wit my goonies we be w...,"[you, can, catch, me, rollin, wit, my, goonies..."
2,2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena'],[],[freeman-dwamena],[freeman-dwamena],freeman-dwamena-ananseem-stories-annotated,there is in ghana story-feeling art called ana...,"[there, is, in, ghana, story-feeling, art, cal..."
3,3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky'],[],[kazaky],[kazaky],kazaky-the-sun,you dont see me now i am far away you dont fee...,"[you, dont, see, me, now, i, am, far, away, yo..."
4,4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...,[],[jimmy-the-rapper-and-jay-bel],[jimmy-the-rapper-and-jay-bel],jimmy-the-rapper-and-jay-bel-amused,came came saw saw conquered your body.. frame ...,"[came, came, saw, saw, conquered, your, body....."
5,5,358482,Coal Town Road,https://genius.com/Ryans-fancy-coal-town-road-...,"['[Chorus]', 'We get up in the black down the ...","[('Release Date', [])]",['https://genius.com/artists/Ryans-fancy'],[],[ryans-fancy],[ryans-fancy],ryans-fancy-coal-town-road,we get up in the black down the coal town road...,"[we, get, up, in, the, black, down, the, coal,..."
6,6,358477,Tworzymy Historię,https://genius.com/Jwp-bc-tworzymy-historie-ly...,"['[Verse 1: Ero/Kosi/Łysol]', 'Światła, kamera...","[('Written By', ['https://genius.com/artists/A...",['https://genius.com/artists/Jwp-bc'],"[ajzol, kosi, ero-jwp, dj-falcon1]",[jwp-bc],"[ajzol, kosi, ero-jwp, dj-falcon1]",jwp-bc-tworzymy-historie,światła kamera akcja sprawdź to ziom to krąg j...,"[światła, kamera, akcja, sprawdź, to, ziom, to..."
7,7,358503,Paradiso: Canto 29,https://genius.com/Dante-alighieri-paradiso-ca...,"['Paradiso: Canto XXIX', 'At what time both th...",[],['https://genius.com/artists/Dante-alighieri'],[],[dante-alighieri],[dante-alighieri],dante-alighieri-paradiso-canto-29-annotated,paradiso canto xxix at what time both the chil...,"[paradiso, canto, xxix, at, what, time, both, ..."
8,8,358506,Filly Dreams,https://genius.com/Jeff-burgess-and-the-bad-ma...,"['Are you scared of the way things change', ""W...","[('Release Date', [])]",['https://genius.com/artists/Jeff-burgess-and-...,[],[jeff-burgess-and-the-bad-mares],[jeff-burgess-and-the-bad-mares],jeff-burgess-and-the-bad-mares-filly-dreams,are you scared of the way things change when t...,"[are, you, scared, of, the, way, things, chang..."
9,9,358485,Never Forget (5 Years Later),https://genius.com/Lozo-never-forget-5-years-l...,"[""We're constantly emboldened to neVer forgeT ...",[],['https://genius.com/artists/Lozo'],[],[lozo],[lozo],lozo-never-forget-5-years-later-annotated,were constantly emboldened to never forget 32 ...,"[were, constantly, emboldened, to, never, forg..."


In [28]:
df.to_csv("million.9_song.csv", index=False)

In [30]:
df.shape

(1844757, 12)

In [7]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [8]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df["lyric_list"])]

In [9]:
documents[0:10]

[TaggedDocument(words="['ive', 'got', 'nowhere', 'left', 'to', 'hide', 'no', 'time', 'left', 'for', 'a', 'compromise', 'do', 'i', 'push', 'back', 'harder', 'than', 'i', 'have', 'before', 'or', 'just', 'fall', 'if', 'i', 'cant', 'decide', 'that', 'danger', 'is', 'real', 'i', 'feel', 'it', 'all', 'the', 'time', 'so', 'i', 'wear', 'it', 'in', 'the', 'name', 'to', 'remind', 'me', 'why', 'everything', 'i', 'gave', 'up', 'or', 'i', 'left', 'behind', 'just', 'to', 'be', 'where', 'i', 'am', 'tonight', 'yeah', 'but', 'you', 'cant', 'touch', 'me', 'i', 'cant', 'be', 'touched', 'you', 'cant', 'hold', 'me', 'the', 'weights', 'too', 'much', 'you', 'cant', 'buy', 'me', 'i', 'have', 'no', 'price', 'i', 'have', 'no', 'backup', 'i', 'have', 'no', 'crutch', 'one', 'last', 'attempt', 'all-out', 'attack', 'and', 'if', 'you', 'push', 'me', 'ill', 'push', 'you', 'back', 'and', 'should', 'i', 'fall', 'youll', 'go', 'down', 'with', 'me', 'make', 'sure', 'you', 'remember', 'that', 'do', 'you', 'have', 'what', 

In [16]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=5)

In [17]:
model.build_vocab(documents)

In [18]:
!pip install cython

You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
model.train(documents, total_examples=model.corpus_count, epochs=5)

In [20]:
pj = "Yamborghini chain, rest in peace to my superior\
Hermès link could feed a village in Liberia\
TMZ taking pictures, causin' mad hysteria\
Momma see me on BET and started tearin' up\
I'ma start killin' niggas, how you get that trife?\
I attended Harlem picnics where you risked your life\
Uncle used to skim work, sellin' nicks at night\
I was only 8 years old, watching Nick at Nite\
Uncle Psycho was in that bathroom buggin'!\
Knife to his guts, hope Daddy don't cut him\
Suicidal thoughts brought to me with no advisory\
He was pitchin' dummy, sellin' fiends mad ivory\
Grandma had the arthritis in her hands, bad!\
She was poppin' pills like rappers in society\
I fuck yo bitch for the irony\
I'll send Meechy at yo ho if yo bitch keep eyein' me"

pj = pj.split(" ")
pj = model.infer_vector(pj)
x = model.docvecs.most_similar([pj],topn=10)

In [21]:
l = [x[i][0] for i in range(len(x))] 

In [22]:
x

[(1017590, 0.7067300081253052),
 (1017363, 0.6883159279823303),
 (852067, 0.6602106690406799),
 (1076349, 0.6504916548728943),
 (894471, 0.6485536098480225),
 (1732427, 0.6448930501937866),
 (1716827, 0.6446694731712341),
 (1635977, 0.6420323252677917),
 (1455145, 0.6391947865486145),
 (994148, 0.6391171216964722)]

In [23]:
df.iloc[l]

,song_num,title,url,lyrics,side_table,header_links,meta_data_1,fixed_header_links,meta_data_2,song_identifier,clean_lyrics,lyric_list
1017590,119548,Flipping Da Bird,https://genius.com/Millionaire-thought-flippin...,"['VERSE 1: Joey Merritt', 'Verse 2: (Millionai...",[],['https://genius.com/artists/Millionaire-thoug...,[],['millionaire-thought'],['millionaire-thought'],millionaire-thought-flipping-da-bird,verse 1 joey merritt verse 2 millionaire thoug...,"['verse', '1', 'joey', 'merritt', 'verse', '2'..."
1017363,119319,Girl,https://genius.com/Millzysosa-girl-lyrics,"['[Verse 1: MillzySosa]', 'Uh, & I say ooo ahh...",[],['https://genius.com/artists/Millzysosa'],[],['millzysosa'],['millzysosa'],millzysosa-girl,uh & i say ooo ahhh girl you just make me craz...,"['uh', '&', 'i', 'say', 'ooo', 'ahhh', 'girl',..."
852067,15182,Kronologik,https://genius.com/Cypress-hill-kronologik-lyrics,"['[Hook: Kurupt and B-Real]', ""You don't know ...","[('Written By', ['https://genius.com/artists/B...",['https://genius.com/artists/Cypress-hill'],"['b-real', 'dj-muggs', 'kurupt', 'troy-staton'...",['cypress-hill'],"['b-real', 'dj-muggs', 'kurupt', 'troy-staton'...",cypress-hill-kronologik,you dont know no real shit nigga its the real ...,"['you', 'dont', 'know', 'no', 'real', 'shit', ..."
1076349,534602,The Cycle,https://genius.com/Gordof-the-cycle-lyrics,"['Intro:', ""Every nigga go through a stage, to...","[('Release Date', [])]",['https://genius.com/artists/Gordof'],[],['gordof'],['gordof'],gordof-the-cycle,intro every nigga go through a stage to where ...,"['intro', 'every', 'nigga', 'go', 'through', '..."
894471,104818,Swimming Pools (Smoke),https://genius.com/Lil-jj-swimming-pools-smoke...,"['Verse 1:', 'Look, I be on Coronas', ""I'm by ...",[],['https://genius.com/artists/Lil-jj'],[],['lil-jj'],['lil-jj'],lil-jj-swimming-pools-smoke,verse 1 look i be on coronas im by myself im a...,"['verse', '1', 'look', 'i', 'be', 'on', 'coron..."
1732427,1991099,Panic Station (Alternate Version Mixed by Madeon),https://genius.com/Muse-panic-station-alternat...,"['[Verse 1]', ""You won't get much closer"", 'Ti...","[('Release Date', []), ('Remix Of', ['https://...",['https://genius.com/artists/Muse'],['muse-panic-station'],['muse'],['muse'],muse-panic-station-alternate-version-mixed-by-...,you wont get much closer till you sacrifice it...,"['you', 'wont', 'get', 'much', 'closer', 'till..."
1716827,1529160,Dinner At 6,https://genius.com/Bamboo-dinner-at-6-lyrics,['Dinner at 6 I can no longer wait for your ca...,"[('Written By', ['https://genius.com/artists/B...",['https://genius.com/artists/Bamboo'],"['bamboo', 'bamboo-manalac', 'nathan-azarcon',...",['bamboo'],"['bamboo', 'bamboo-manalac', 'nathan-azarcon',...",bamboo-dinner-at-6,dinner at 6 i can no longer wait for your call...,"['dinner', 'at', '6', 'i', 'can', 'no', 'longe..."
1635977,801049,Depraved,https://genius.com/Anberlin-depraved-lyrics,"['Are you depraved', 'Or are you deceived?', '...","[('Written By', ['https://genius.com/artists/S...",['https://genius.com/artists/Anberlin'],"['stephen-christian', 'joseph-milligan', 'deon...",['anberlin'],"['stephen-christian', 'joseph-milligan', 'deon...",anberlin-depraved,are you depraved or are you deceived excuses a...,"['are', 'you', 'depraved', 'or', 'are', 'you',..."
1455145,459022,Evil Tendencies 2,https://genius.com/Nick-xaman-evil-tendencies-...,"['[VERSE ONE]', '(Nick Xaman)', 'Chillin like ...",[],['https://genius.com/artists/Nick-xaman'],[],['nick-xaman'],['nick-xaman'],nick-xaman-evil-tendencies-2,nick xaman chillin like a villain yeah im evil...,"['nick', 'xaman', 'chillin', 'like', 'a', 'vil..."
994148,168224,The World’s Wanderers,https://genius.com/Percy-bysshe-shelley-the-wo...,"['[Published by Mrs. Shelley, ""Posthumous Poem...",[],['https://genius.com/artists/Percy-bysshe-shel...,[],['percy-bysshe-shelley'],['percy-bysshe-shelley'],percy-bysshe-shelley-the-worlds-wanderers-anno...,1. tell me thou star whose wings of light spee...,"['1.',

In [24]:
model.save("lyric_doc2vec")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [25]:
model2=Doc2Vec.load("lyric_doc2vec")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
df.to_csv("million.9_song.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("million.9_song.csv")

In [4]:
df.head()

,song_num,title,url,lyrics,side_table,header_links
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"[""I've got nowhere left to hide"", 'No time lef...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids']
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"['[Hook]', 'You can catch me rollin wit my goo...","[('Release Date', [])]",['https://genius.com/artists/Caution']
2,358464,“Ananseem Stories”,https://genius.com/Freeman-dwamena-ananseem-st...,['There is in Ghana story-feeling art called A...,"[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena']
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"['[Verse 1]', ""You don't see me"", 'Now I am fa...","[('Release Date', [])]",['https://genius.com/artists/Kazaky']
4,358510,Amused,https://genius.com/Jimmy-the-rapper-and-jay-be...,"['[Verse 1 - Jimmy The Rapper]', 'Came came, s...","[('Release Date', [])]",['https://genius.com/artists/Jimmy-the-rapper-...


##  Bag of Words vectorization for song meta data

Networkx graph is coming out way too small. I have over 9 million edges. But the graph has 55046 nodes?


In [ ]:
from ast import literal_eval
df["meta_data_2"] = df["meta_data_2"].apply(literal_eval)

In [ ]:
data_strings = [" ".join(i) for i in df.meta_data_2]

In [ ]:
data_strings

In [ ]:
data_vocabulary = [ i for j in df.meta_data_2 for i in j ]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
)

In [ ]:
vectorizer.vocabulary =set(data_vocabulary)

In [ ]:
X = vectorizer.fit_transform(data_strings)

In [ ]:
X

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300, n_iter=30, random_state=42)

In [ ]:
x_svd = svd.fit_transform(X)

In [ ]:
x_svd

In [1]:
import numpy

In [2]:
numpy.save("x_svd",x_svd)

NameError: name 'x_svd' is not defined

In [3]:
vv = numpy.load("lyric_doc2vec.docvecs.vectors_docs.npy")
x_svd=numpy.load("x_svd.npy")

In [4]:
full_data = numpy.concatenate((vv,x_svd), axis=1)

In [5]:
numpy.save("full_data",full_data)

In [6]:
!pip install annoy

    100% |████████████████████████████████| 645kB 27.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for annoy ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/78/3a/1c913a402f4d84f68eb0ea4f7c0de0d392725415c8ebe51068
Successfully built annoy
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
from annoy import AnnoyIndex
import random

f = 200
t = AnnoyIndex(f)

In [10]:
for i in range(len(full_data)):
    t.add_item(i,full_data[i])

In [11]:
t.build(20) # 10 trees
t.save('test.ann')


True

In [12]:
l = t.get_nns_by_item(15, 10, search_k=-1, include_distances=False)

In [13]:
df.iloc[l]

NameError: name 'df' is not defined